# Importation des dependances 

In [114]:
import pandas as pd

# Mission 
Notre objectif dans cette première phase du projet consiste à charger le jeu de données disponible sur Open Food Fac et prendre connaissance du format des données dans un premier temps. Dans un second temps, nous examinons la qualité des données et ensuite apporter des actions correctives afin de la préparer pour la seconde phase, **phase d'exploration !**

## 1 - Chargement des données
Nous avons télecharger le jeu de données disponible [ici](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/fr.openfoodfacts.org.products.csv.zip) puis nous l'avons chargé à l'aide de la founction `read_csv` de `pandas`. Le résultat du chaegement est stocké dans la variable `df_foods`.

Comme l'indique le [fichier descriptif des données](https://world.openfoodfacts.org/data/data-fields.txt):
- les colonnes sont uniquement separarées par des tabulations
- l'encodage utilisé est *UTF-8*
- les entêtes des colonnes sont déjà définis 
Ces informations nous permettent de définir les arguments des la fonction `read_csv`

In [115]:
df_foods = pd.read_csv('data/foods.csv', sep = '\t', encoding = 'UTF-8')

/opt/miniconda3/envs/p1Sante/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


**👉  Observons les 5 premières lignes de notre dataframe pour verifier si les données sont correctements chargées**

In [116]:
df_foods.head(5)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,3087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,NaN,NaN
2,4559,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
3,16087,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
4,16094,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055653,2017-03-09T10:34:13Z,1489055653,2017-03-09T10:34:13Z,Organic Polenta,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 🧠 Comme vous pouvez le constater, notre chargement est correct parce que toutes les colonnes attendues existent. ####  

## 2 - Le DataFrame

**👉 Voyons combien de `lignes` et de `colonnes` sont contenues dans `df_foods`**

In [117]:
df_foods.shape

(320772, 162)

In [118]:
df_foods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Columns: 162 entries, code to water-hardness_100g
dtypes: float64(106), object(56)
memory usage: 396.5+ MB


#### 🧠 Nous avons 56 colonnes de `type` objet qui correspodent à des chaînes de caractères. Le jeu de données occupe presque `400 MB de la mémoire RAM`. Ce qui est considérable. Mais il reste à savoir si tous ces colonnes aussi riches. Pensons aux **données manquantes!**. Il va falloir identifier les colonnes qui nous seront utiles à notre application par la suite et se debarasser des autres. ####

## 3 - Qualité des données

**👉 Mesurons le pourcentage de `valeurs manquantes` par colonne**

In [119]:
df_foods.isna().mean(axis=0)

code                       0.000072
url                        0.000072
creator                    0.000006
created_t                  0.000009
created_datetime           0.000028
                             ...   
carbon-footprint_100g      0.999165
nutrition-score-fr_100g    0.310382
nutrition-score-uk_100g    0.310382
glycemic-index_100g        1.000000
water-hardness_100g        1.000000
Length: 162, dtype: float64

#### 🧠 Nous pouvons constater les valeurs pour certaines colonnes sont quasi inéxistantes. Nous les supprimerons dans la partie nettoyages. ####

**👉 Vérifions si les colonnes `date` sont bien typées**

In [120]:
df_foods[['created_t','created_datetime','last_modified_t',	'last_modified_datetime']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Data columns (total 4 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   created_t               320769 non-null  object
 1   created_datetime        320763 non-null  object
 2   last_modified_t         320772 non-null  object
 3   last_modified_datetime  320772 non-null  object
dtypes: object(4)
memory usage: 9.8+ MB


#### 🧠 Comme nous pouvons le voir, les dates ne sont pas bien typées. Nous corrigerons ce problème dans la partie nettoyage celle format. ####

**👉 Voyons s'il existe des duplicatas**

In [121]:
df_foods.duplicated(subset=['code']).sum()

133

#### 🧠 Vue que les duplicatas n'ont pas de valeur ajouté, il va falloir les supprimer pour reduire la taille du jeu de données. ####

## 4 - Nettoyage du jeu de données

**👉 Supprimons les duplicatas**

In [122]:
df_foods = df_foods.drop_duplicates(subset=['code'])

In [123]:
df_foods.duplicated(subset=['code']).sum()

0

**👉 Supprimons toutes les colonnes avec tous les champs inconnus, ou `NaN` en anglais**

In [124]:
df_foods = df_foods.loc[:, ~df_foods.isnull().all()]

In [125]:
df_foods = df_foods.drop(columns=[col for col in df_foods.columns if not df_foods[col].isna().mean()<0.9], axis=1)

In [126]:
df_foods

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,fiber_100g,proteins_100g,salt_100g,sodium_100g,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
0,3087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,...,3.6,3.57,0.00000,0.000,0.0,0.0214,0.000,0.00129,14.0,14.0
2,4559,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,NaN,NaN,...,7.1,17.86,0.63500,0.250,0.0,0.0000,0.071,0.00129,0.0,0.0
3,16087,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,NaN,NaN,...,7.1,17.86,1.22428,0.482,NaN,NaN,0.143,0.00514,12.0,12.0
4,16094,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055653,2017-03-09T10:34:13Z,1489055653,2017-03-09T10:34:13Z,Organic Polenta,NaN,NaN,...,5.7,8.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320767,9948282780603,http://world-fr.openfoodfacts.org/produit/9948...,openfoodfacts-contributors,1490631299,2017-03-27T16:14:59Z,1491244498,2017-04-03T18:34:58Z,Tomato & ricotta,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320768,99567453,http://world-fr.openfoodfacts.org/produit/9956...,usda-ndb-import,1489059076,2017-03-09T11:31:16Z,1491244499,2017-04-03T18:34:59Z,"Mint Melange Tea A Blend Of Peppermint, Lemon ...",NaN,NaN,...,0.0,0.00,0.00000,0.000,0.0,0.0000,0.000,0.00000,0.0,0.0
320769,9970229501521,http://world-fr.openfoodfacts.org/produit/9970...,tomato,1422099377,2015-01-24T11:36:17Z,1491244499,2017-04-03T18:34:59Z,乐吧泡菜味薯片,Leba pickle flavor potato chips,50 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320770,9980282863788,http://world-fr.openfoodfacts.org/produit/9980...,openfoodfacts-contributors,1492340089,2017-04-16T10:54:49Z,1492340089,2017-04-16T10:54:49Z,Tomates aux Vermicelles,NaN,67g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## To do 
1 - Description metiers
2 - Visual pour presenter les valeurs misingo
3 - install plotly
4 - Univarier des colonnes selectionnés


**👉 Mainténant il serait utile d'avoir quelques informations sur le `type` de chacune des `colonnes`**